In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install -q huggingface_hub
!pip install -q transformers
!pip install -q peft
!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -q datasets
!pip install -q trl # Для упрощения SFT (Supervised Fine-tuning)

In [ ]:
from huggingface_hub import login
import json
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer, SFTConfig

In [ ]:
def convert_redial_conversation_to_messages(conversation_dict):
    """
    Преобразует словарь одного диалога из ReDIAL в формат {"messages": [...]}
    """
    messages = []
    initiator_id = conversation_dict['initiatorWorkerId']
    respondent_id = conversation_dict['respondentWorkerId']

    for msg in conversation_dict['messages']:
        sender_role = "user" if msg['senderWorkerId'] == initiator_id else "assistant"
        messages.append({
            "role": sender_role,
            "content": msg['text']
        })
    return {"messages": messages}


In [ ]:
def load_and_process_redial(file_path):
    processed_data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            conv_dict = json.loads(line)
            processed_data.append(convert_redial_conversation_to_messages(conv_dict))
    return Dataset.from_list(processed_data)

In [ ]:
train_dataset = load_and_process_redial('/content/drive/MyDrive/Colab Notebooks/datasets/train.jsonl')
val_dataset = load_and_process_redial('/content/drive/MyDrive/Colab Notebooks/datasets/valid.jsonl')
test_dataset = load_and_process_redial('/content/drive/MyDrive/Colab Notebooks/datasets/test.jsonl')

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
new_model_name = "llama-3.1-8b-redial-qlora"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Квантование в 4 бита
    bnb_4bit_quant_type="nf4", # Тип квантования
    bnb_4bit_compute_dtype=torch.float16, # Тип данных для вычислений
    bnb_4bit_use_double_quant=True, # Использование двойного квантования для экономии памяти
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token # Очень важно для корректной работы

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, # Применяем квантование
    device_map={"": 0}, # Загружаем модель на GPU 0 (T4)
    torch_dtype=torch.float16, # Устанавливаем тип данных
    attn_implementation="eager" # Иногда помогает с памятью, можно попробовать "flash_attention_2" если доступно
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
peft_config = LoraConfig(
    r=16,  # Ранг LoRA (меньше - быстрее и экономнее, больше - мощнее)
    lora_alpha=16, # Масштабирующий фактор
    lora_dropout=0.1, # Dropout для регуляризации
    bias="none", # Не обучаем bias
    task_type="CAUSAL_LM", # Тип задачи
    target_modules=[ # Модули, к которым применяется LoRA. Эти подходят для LLaMA.
        "q_proj", "k_proj", "v_proj",
        "o_proj", "gate_proj", "up_proj", "down_proj"
    ]
)

In [ ]:
sft_config = SFTConfig(
    output_dir=new_model_name,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=4,
    optim="paged_adamw_32bit",
    save_steps=1000,
    logging_steps=100,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="none",
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    load_best_model_at_end=False,
    max_length=128,
    packing=False,
    # -------------------------------
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
)

In [ ]:
import pkg_resources
libraries = ['transformers', 'trl', 'peft', 'torch', 'datasets', 'accelerate', 'bitsandbytes', 'huggingface_hub']

for lib in libraries:
    try:
        version = pkg_resources.get_distribution(lib).version
        print(f"{lib} version: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{lib} is not installed")

transformers version: 4.54.0
trl version: 0.20.0
peft version: 0.16.0
torch version: 2.6.0+cu124
datasets version: 4.0.0
accelerate version: 1.9.0
bitsandbytes version: 0.46.1
huggingface_hub version: 0.34.1


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    args=sft_config,   # Все параметры обучения передаются здесь
    formatting_func=None,      # Опционально, если нужна кастомная обработка
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/8004 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/8004 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2002 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/2002 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=251, training_loss=1.2810168019328934, metrics={'train_runtime': 5602.6288, 'train_samples_per_second': 1.429, 'train_steps_per_second': 0.045, 'total_flos': 4.639113920013926e+16, 'train_loss': 1.2810168019328934})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/lora_weights_llama3_rec")
tokenizer.save_pretrained("/content/drive/MyDrive/lora_weights_llama3_rec")

NameError: name 'model' is not defined